In [1]:
import os
import pywt
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig

In [23]:
# files = os.listdir("spectra")

# for filename in files:
#     np.genfromtxt(filename, delimiter=",")

limit_low = None
limit_high = None

spectrum = np.genfromtxt("spectra/E (1).TXT", delimiter=",")
spectrum = np.transpose(spectrum)
wavenumbers = spectrum[0]
intensities = spectrum[1]

if limit_low is not None:
    limit_low_index = list(wavenumbers).index(limit_low)
else:
    limit_low_index = 0

if limit_high is not None:
    limit_high_index = list(wavenumbers).index(limit_high)
else:
    limit_high_index = len(wavenumbers)

wavenumbers = wavenumbers[limit_low_index:limit_high_index]
intensities = intensities[limit_low_index:limit_high_index]

In [38]:
intensities_sg = sig.savgol_filter(intensities, 
                                   window_length=3, 
                                   polyorder=0, 
                                   deriv=0)

In [39]:
%matplotlib
fig, ax = plt.subplots()
ax.plot(wavenumbers, intensities_sg, linewidth = 1, color = "red")
ax.set_xlim(limit_low, limit_high)
ax.invert_xaxis()
ax.grid()

Using matplotlib backend: Qt5Agg


In [52]:
intensities_ext = np.concatenate((np.full(400, intensities_sg[0]), intensities_sg, np.full(400, intensities_sg[-1])))
scales = np.arange(1,201)

coefs, freqs = pywt.cwt(data=intensities_ext, scales=scales, wavelet="mexh")

coefs = coefs[:,400:-400]

6821

In [53]:
wavenumbers_grid, scales_grid = np.meshgrid(wavenumbers, scales)

In [54]:
%matplotlib
fig, ax = plt.subplots()
cp = ax.contourf(wavenumbers_grid, scales_grid, coefs)
fig.colorbar(cp)

Using matplotlib backend: Qt5Agg


In [57]:
p_coefs = coefs / coefs.sum()

shannon_H = np.sum(p_coefs * np.log(p_coefs))

<ipython-input-57-fe72b2a3c0b7>:3: RuntimeWarning: invalid value encountered in log
  shannon_H = np.sum(p_coefs * np.log(p_coefs))


Call signature:  np.log(*args, **kwargs)
Type:            ufunc
String form:     <ufunc 'log'>
File:            c:\users\daniel\.conda\envs\raman\lib\site-packages\numpy\__init__.py
Docstring:      
log(x, /, out=None, *, where=True, casting='same_kind', order='K', dtype=None, subok=True[, signature, extobj])

Natural logarithm, element-wise.

The natural logarithm `log` is the inverse of the exponential function,
so that `log(exp(x)) = x`. The natural logarithm is logarithm in base
`e`.

Parameters
----------
x : array_like
    Input value.
out : ndarray, None, or tuple of ndarray and None, optional
    A location into which the result is stored. If provided, it must have
    a shape that the inputs broadcast to. If not provided or None,
    a freshly-allocated array is returned. A tuple (possible only as a
    keyword argument) must have length equal to the number of outputs.
where : array_like, optional
    This condition is broadcast over the input. At locations where the
    condi